## Realizar optimizacion con optuna pero de todo el pipeline, desde imputacion, tolerancia, encoding hasta el modelo

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from feature_engine.imputation import (AddMissingIndicator, 
                                       MeanMedianImputer, 
                                       CategoricalImputer
                                       )
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder, OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import optuna
import lightgbm as lgb
from optuna.visualization import (plot_param_importances,
                                  plot_optimization_history,
                                  plot_parallel_coordinate,
                                  plot_slice,
                                  plot_contour)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error


In [30]:
data_gold = pd.read_excel("../data/gold/car_gold.xlsx")
## selected feature
features = ["price", "antiguedad", "kilometraje", "vehicle_brand", "vehicle_line", 'location_state']
data_gold = data_gold[features]
data_gold.head()



,price,antiguedad,kilometraje,vehicle_brand,vehicle_line,location_state
0,183900000,0.0,5800.0,byd,song,bogota_d.c.
1,64900000,8.0,59000.0,mazda,3,antioquia
2,72000000,6.0,47000.0,NaN,NaN,bogota_d.c.
3,59500000,0.0,3600.0,kia,picanto,bogota_d.c.
4,63000000,9.0,81000.0,nissan,x-trail,bogota_d.c.


In [31]:
## dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(data_gold.drop(labels=['price'], axis=1),
                                                    data_gold['price'],
                                                    test_size=0.3,
                                                    random_state=0) # reproducibilidad

In [32]:
def autoconfigure_features(data):
    vars_with_na = [var for var in data.columns if data[var].isnull().sum() > 0]
    num_vars = data.select_dtypes(include=['int64', 'float64']).columns
    num_vars_na = [var for var in num_vars if var in vars_with_na]
    cat_vars = data.select_dtypes(include=['object']).columns
    cat_vars_na = [var for var in cat_vars if var in vars_with_na]

    output = {
        "num_vars": num_vars_na,
        "cat_vars": cat_vars_na,
        "vars_with_na": vars_with_na,
        "num_vars_na": num_vars_na,
        "cat_vars_na": cat_vars_na
    }
    return output


In [33]:
variables = autoconfigure_features(data_gold)

In [34]:
num_vars_na = variables['num_vars_na']
cat_vars_na = variables['cat_vars_na']
vars_with_na = variables['vars_with_na']
num_vars = variables['num_vars']
cat_vars = variables['cat_vars']


In [50]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "mse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 150, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.008, 0.02, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 500, 800),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 0.9),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 10.0, log=True),
    }
    imputation_method_suggest = trial.suggest_categorical('imputation_method', ['median', 'mean'])
    rarelabel_tol_suggest = trial.suggest_float('rarelabel_tol', 0.0009, 0.005)
    encoding_method_suggest = trial.suggest_categorical('encoding_method', ['ordered', 'onehot'])
    scaler_method_suggest = trial.suggest_categorical('scaler_method', ['standard', 'minmax'])
    pipeline_steps = [
        ("missing_indicator", AddMissingIndicator(variables=vars_with_na)),
        ("numerical_imputation", MeanMedianImputer(imputation_method=imputation_method_suggest, variables=num_vars_na)),
        ("categorical_imputation", CategoricalImputer(imputation_method='missing', fill_value='missing', variables=cat_vars_na)),
        ("rare_label_encoder", RareLabelEncoder(tol=rarelabel_tol_suggest, variables=cat_vars, n_categories=1)),
    ]
    if encoding_method_suggest == 'ordered':
        pipeline_steps.append(("ordinal_encoder", OrdinalEncoder(variables=cat_vars, encoding_method='ordered')))
    else:
        pipeline_steps.append(("onehot_encoder", OneHotEncoder(variables=cat_vars, drop_last=True)))
    
    if scaler_method_suggest == 'standard':
        pipeline_steps.append(("scaler", SklearnTransformerWrapper(transformer=StandardScaler())))
    else:
        pipeline_steps.append(("scaler", SklearnTransformerWrapper(transformer=MinMaxScaler())))
    
    pipeline_steps.append(("model", lgb.LGBMRegressor(**params)))
    
    pipeline = Pipeline(pipeline_steps)
    pipeline.fit(X_train, y_train)
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    r2 = r2_score(y_test, y_pred_test)
    return r2
    
    

In [52]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2025-10-23 19:31:40,291] A new study created in memory with name: no-name-cd681a48-a985-4942-acbf-c8f019d610d5


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-23 19:31:52,535] Trial 0 finished with value: 0.6663049353358369 and parameters: {'num_leaves': 216, 'max_depth': 13, 'learning_rate': 0.013382038116433721, 'n_estimators': 620, 'min_child_samples': 5, 'subsample': 0.7637401090547913, 'colsample_bytree': 0.7529888953517473, 'reg_alpha': 0.10477390911337738, 'reg_lambda': 0.13555474794273625, 'imputation_method': 'mean', 'rarelabel_tol': 0.002435718543349576, 'encoding_method': 'onehot', 'scaler_method': 'minmax'}. Best is trial 0 with value: 0.6663049353358369.
[I 2025-10-23 19:31:54,647] Trial 1 finished with value: 0.603977412554102 and parameters: {'num_leaves': 295, 'max_depth': 4, 'learning_rate': 0.014217797920231107, 'n_estimators': 772, 'min_child_samples': 8, 'subsample': 0.9841161571097934, 'colsample_bytree': 0.7564992070292442, 'reg_alpha': 0.6971129291494439, 'reg_lambda': 0.010290817137363501, 'imputation_method': 'median', 'rarelabel_tol': 0.0041825239021547125, 'encoding_method': 'onehot', 'scaler_method': 'm

In [54]:
study.best_params

{'num_leaves': 186,
 'max_depth': 7,
 'learning_rate': 0.008817351759028794,
 'n_estimators': 797,
 'min_child_samples': 7,
 'subsample': 0.6548106635253633,
 'colsample_bytree': 0.7636929152700445,
 'reg_alpha': 9.371509884255788,
 'reg_lambda': 0.00021761147730702814,
 'imputation_method': 'mean',
 'rarelabel_tol': 0.003270088646746995,
 'encoding_method': 'ordered',
 'scaler_method': 'standard'}

In [55]:
study.best_value


0.6890609208805107

In [57]:
# Filtrar solo los parámetros del modelo LightGBM (quitar los de preprocesamiento)
model_param_keys = [
    'num_leaves', 'max_depth', 'learning_rate', 'n_estimators',
    'min_child_samples', 'subsample', 'colsample_bytree',
    'reg_alpha', 'reg_lambda'
]
best_model_params = {k: v for k, v in study.best_trial.params.items() if k in model_param_keys}
best_model_params


{'num_leaves': 186,
 'max_depth': 7,
 'learning_rate': 0.008817351759028794,
 'n_estimators': 797,
 'min_child_samples': 7,
 'subsample': 0.6548106635253633,
 'colsample_bytree': 0.7636929152700445,
 'reg_alpha': 9.371509884255788,
 'reg_lambda': 0.00021761147730702814}

In [58]:
pipeline_steps = [
    ("missing_indicator", AddMissingIndicator(variables=vars_with_na)),
    ("numerical_imputation", MeanMedianImputer(imputation_method = best_pipeline['imputation_method'], variables=num_vars_na)),
    ("categorical_imputation", CategoricalImputer(imputation_method='missing', fill_value='missing', variables=cat_vars_na)),
    ("rare_label_encoder", RareLabelEncoder(tol=best_pipeline['rarelabel_tol'], variables=cat_vars, n_categories=1)),
    ("ordinal_encoder", OrdinalEncoder(variables=cat_vars, encoding_method='ordered')),
    ("scaler", SklearnTransformerWrapper(transformer=StandardScaler())),
    ("model", lgb.LGBMRegressor(**best_model_params))
]


In [59]:
final_pipeline = Pipeline(pipeline_steps)

In [ ]:
final_pipeline.fit(X_train, y_train)
y_pred_train_lgb = final_pipeline.predict(X_train)
r2_lgb = r2_score(y_train, y_pred_train_lgb)
mse_lgb = np.sqrt(mean_squared_error(y_train, y_pred_train_lgb))
mae_lgb = mean_absolute_error(y_train, y_pred_train_lgb)
medae_lgb = median_absolute_error(y_train, y_pred_train_lgb)
metrics_lgb_train = {
    "r2": r2_lgb,
    "mse": mse_lgb,
    "mae": mae_lgb,
    "medae": medae_lgb
}

y_pred_test_lgb = final_pipeline.predict(X_test)
r2_lgb = r2_score(y_test, y_pred_test_lgb)
mse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_test_lgb))
mae_lgb = mean_absolute_error(y_test, y_pred_test_lgb)
medae_lgb = median_absolute_error(y_test, y_pred_test_lgb)
metrics_lgb_test = {
    "r2": r2_lgb,
    "mse": mse_lgb,
    "mae": mae_lgb,
    "medae": medae_lgb
}
metrics_lgb = {
    "train": metrics_lgb_train,
    "test": metrics_lgb_test
}
metrics_lgb